# Load Libraries

In [1]:
# numpy and pandas
import numpy as np
import pandas as pd
import math

# Generic
import os
import matplotlib.pyplot as plt
import sys

# Images
from PIL import Image
from skimage.transform import resize
import talos as ta
import scikitplot as skplt

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, plot_confusion_matrix, roc_curve

# Tensorflow
import tensorflow as tf

# Keras
from keras.layers import Input, Dense, Dropout
from keras.utils import print_summary
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K


from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

from dcgan import DCGAN
from dcgan_ext import DCGAN


Using TensorFlow backend.


# Global Paths

In [ ]:
path_root = '/home/ygala/TFM_UOC/'
path_images = os.path.join(path_root, 'data', 'covid_images')
path_dcgan = os.path.join(path_images, 'dcgan')

# Global Parameters

In [ ]:
perc_split = [0.7, 0.3, 0.0]
input_shape = (224, 224, 3)
seed = 1234
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Hyperparameters
batch_size = 64
epochs = 1000
learning_rate = 0.0001

# Load Classes and Functions

In [5]:
# chexNet weights

chexnet_weights = '/home/ygala/TFM_UOC/scripts/chexnet/best_weights.h5'

def chexnet_preprocess_input(value):
    return preprocess_input(value)


def get_chexnet_model():
    input_shape = (224, 224, 3)
    img_input = Input(shape=input_shape)
    base_weights = 'imagenet'

    # create the base pre-trained model
    base_model = DenseNet121(
        include_top=False,
        input_tensor=img_input,
        input_shape=input_shape,
        weights=base_weights,
        pooling='avg'
    )

    x = base_model.output
    # add a logistic layer -- let's say we have 14 classes
    predictions = Dense(
        14,
        activation='sigmoid',
        name='predictions')(x)

    # this is the model we will use
    model = Model(
        inputs=img_input,
        outputs=predictions,
    )

    # load chexnet weights
    model.load_weights(chexnet_weights)

    # return model
    return base_model, model

### Auxiliary Functions

In [6]:
### funciones

def get_class_weight(csv_file_path, target_class, learning_rate):
    df = pd.read_csv(csv_file_path, sep=';')
    total_counts = df.shape[0]
    class_weight = []

    ratio_pos = df.loc[(df[target_class] == 'Y')].shape[0] / total_counts
    ratio_neg = df.loc[(df[target_class] == 'N')].shape[0] / total_counts
    class_weight = np.array((ratio_pos, ratio_neg))
        
    return class_weight


def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


def auc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
    return value

def get_model(learning_rate):
    # get base model, model
    base_model, chexnet_model = get_chexnet_model()
    # print a model summary
    # print_summary(base_model)

    x = base_model.output
    # Dropout layer
    #x = Dropout(0.2)(x)
    # one more layer (relu)
    x = Dense(512, activation='relu')(x)
    # Dropout layer
    #x = Dropout(0.2)(x)
    #x = Dense(256, activation='relu')(x)
    # Dropout layer
    #x = Dropout(0.2)(x)
    # add a logistic layer -- let's say we have 6 classes
    predictions = Dense(
        1,
        activation='sigmoid')(x)

    # this is the model we will use
    model = Model(
        inputs=base_model.input,
        outputs=predictions,
    )

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all base_model layers
    for layer in base_model.layers:
        layer.trainable = False

    # initiate an Adam optimizer
    opt = Adam(
        lr=learning_rate,
        beta_1=0.9,
        beta_2=0.999,
        decay=0.0,
        amsgrad=False
    )
    
    #K.clear_session()
    # Let's train the model using Adam
    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=[auc])
    #K.clear_session()
    
    return base_model, model



### Adversarial Generative Networks functions

In [9]:
class DCGAN():
    def __init__(self,img_rows,img_cols,channels,latent_dim=100):
        # Input shape

        self.latent_dim = latent_dim

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        rr = int(round(self.img_rows/4))
        cc = int(round(self.img_cols/4))
        model.add(Dense(3 * rr * cc, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((rr, cc, 3)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self,X_train, epochs, name='aux', batch_size=128, save_interval=50):

        self.name = name
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/" + self.name + "_%d.png" % epoch)
        plt.close()




In [10]:
model = DCGAN(224, 224, 1)
#(224, 224, 3)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 32)      320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 57, 57, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 57, 57, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 57, 57, 64)       

# Load Data

## Read data

In [9]:
case_features = pd.read_csv(os.path.join(path_images, 'clean_data.csv'), sep=';');
case_features

,patientid,index,SpecificCharacterSet,SOPClassUID,SOPInstanceUID,StudyDate,StudyTime,AccessionNumber,Modality,ConversionType,...,PixelSpacing,BitsAllocated,BitsStored,HighBit,PixelRepresentation,LossyImageCompression,LossyImageCompressionMethod,PixelData,filename,survival
0,1,362424,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202004071536250109,20200407,153625.000,ACC00009757,CR,NaN,...,NaN,16,12,11,0,0,NaN,Array of 5549966 elements,362424.png,N
1,13,161101,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003210459280593,20200321,45928.000,ACC00005107,CR,NaN,...,NaN,16,12,11,0,0,NaN,Array of 6069570 elements,161101.png,Y
2,13,161102,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003260728100281,20200326,72810.000,ACC00005301,CR,NaN,...,NaN,16,12,11,0,0,NaN,Array of 5683394 elements,161102.png,Y
3,13,161103,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003220726220593,20200322,72622.000,ACC00005441,CR,NaN,...,NaN,16,12,11,0,0,NaN,Array of 5989198 elements,161103.png,Y
4,13,161104,ISO_IR 100,Computed Radiography Image Storage,1.3.12.2.1107.5.3.56.2693.11.202003221418280437,20200322,141828.000,ACC00005552,CR,NaN,...,NaN,16,12,11,0,0,NaN,Array of 5990704 elements,161104.png,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,2278,460652,ISO_IR 100,Digital X-Ray Image Storage - For Presentation,1.3.46.670589.30.36.0.1.18774111139.1586556014...,20200410,225716.391,ACC00008725,DX,NaN,...,NaN,16,12,11,0,0,NaN,Array of 5674014 elements,460652.png,Y
3481,2280,461518,NaN,Computed Radiography Image Storage,1.3.51.0.7.11525005549.51456.28486.35726.61854...,20200415,103742.000,ACC00010093,CR,NaN,...,"[0.15, 0.15]",16,12,11,0,0,NaN,Array of 5694582 elements,461518.png,Y
3482,2286,466280,ISO_IR 100,Computed Radiography Image Storage,1.3.51.0.7.3142572467.2116.61512.35199.56756.9...,20200414,181443.000,ACC00009959,CR,NaN,...,"[0.15, 0.15]",16,12,11,0,0,NaN,Array of 6762522 elements,466280.png,Y
3483,2287,466281,ISO_IR 100,Computed Radiography Image Storage,1.3.51.0.7.12387810049.31749.41031.46011.30246...,20200415,95222.000,ACC00009454,CR,NaN,...,"[0.1, 0.1]",16,15,14,0,0,NaN,Array of 19441046 elements,466281.png,Y


In [10]:
case_features.shape

(3485, 38)

## Split data

In [ ]:
target_class = 'survival'
np.random.seed(seed)

### Positive patient ids 
positive = case_features[case_features.survival.values == 'Y']
positive_ids = positive.patientid.unique()

### Negative patient ids 
negative = case_features[case_features.survival.values == 'N']
negative_ids = negative.patientid.unique()
n_negative_ids = len(negative_ids)

### Test
test_size = perc_split[2]*n_negative_ids
test_ids = np.append(np.random.choice(positive_ids, size = math.floor(test_size), replace = False),
                     np.random.choice(negative_ids, size = math.floor(test_size), replace = False))
positive_ids = positive_ids[~np.isin(positive_ids, test_ids)]
negative_ids = negative_ids[~np.isin(negative_ids, test_ids)]

### Val
val_size = perc_split[1]*n_negative_ids
val_ids = np.append(np.random.choice(positive_ids, size = math.floor(val_size), replace = False),
                     np.random.choice(negative_ids, size = math.floor(val_size), replace = False))
positive_ids = positive_ids[~np.isin(positive_ids, val_ids)]
negative_ids = negative_ids[~np.isin(negative_ids, val_ids)]

### Train
train_ids = np.append(positive_ids,
                    negative_ids)

# Split dataset based on patient ids
case_features_train = case_features[case_features.patientid.isin(train_ids)]
case_features_val = case_features[case_features.patientid.isin(val_ids)]
case_features_test = case_features[case_features.patientid.isin(test_ids)]

# Selección del patrón de datos X y del target y
ytrain = case_features_train[target_class]
del case_features_train[target_class]
Xtrain = case_features_train

yval = case_features_val[target_class]
del case_features_val[target_class]
Xval = case_features_val

ytest = case_features_test[target_class]
del case_features_test[target_class]
Xtest = case_features_test

In [ ]:
print(len(case_features_train[ytrain == "Y"].patientid.unique()))
print(len(case_features_train[ytrain == "N"].patientid.unique()))
print(len(case_features_val[yval == "Y"].patientid.unique()))
print(len(case_features_val[yval == "N"].patientid.unique()))
print(len(case_features_test[ytest == "Y"].patientid.unique()))
print(len(case_features_test[ytest == "N"].patientid.unique()))

In [ ]:
print(case_features_train.shape)
print(case_features_val.shape)
print(case_features_test.shape)

In [ ]:
print(ytrain.value_counts())
print(yval.value_counts())
print(ytest.value_counts())

## Read images

### Train

In [ ]:
X_train = []
for i in range(Xtrain.shape[0]):
    print('%0.2f%%' % float(100*i/Xtrain.shape[0]))
    image_path = os.path.join(path_images, 'processed', Xtrain.iloc[i].filename)
    imagen = Image.open(image_path)
    #imagen = np.asarray(imagen.convert("L"))
    #imagen = resize(imagen,  input_shape)
    imagen = np.expand_dims(imagen, axis=-1)
    imagen = resize(imagen,  input_shape)
    X_train.append(imagen)

X_train = np.stack(X_train, axis = 0)
print(X_train.shape)

### Validation

In [ ]:
X_val = []
for i in range(Xval.shape[0]):
    #print('%0.2f%%' % float(100*i/Xval.shape[0]))
    image_path = os.path.join(path_images, 'processed', Xval.iloc[i].filename)
    imagen = Image.open(image_path)
    #imagen = np.asarray(imagen.convert("L"))
    #imagen = resize(imagen,  input_shape)
    imagen = np.expand_dims(imagen, axis=-1)
    imagen = resize(imagen,  input_shape)
    X_val.append(imagen)

X_val = np.stack(X_val, axis = 0)
print(X_val.shape)

### Test

In [ ]:
X_test = []
for i in range(Xtest.shape[0]):
    image_path = os.path.join(path_images, 'processed', Xtest.iloc[i].filename)
    imagen = Image.open(image_path)
    #imagen = np.asarray(imagen.convert("L"))
    #imagen = resize(imagen,  input_shape)
    imagen = np.expand_dims(imagen, axis=-1)
    imagen = resize(imagen,  input_shape)
    X_test.append(imagen)

X_test = np.stack(X_test, axis = 0)
print(X_test.shape)

In [ ]:
np.save('X_test', X_test)
np.save('X_train', X_train)
np.save('X_val', X_val)

In [11]:
#open("filename","w+")
X_test = np.load('X_test.npy')
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')

In [12]:
print(X_val.shape)

(107, 224, 224, 1)


In [ ]:
model.train(X_train, epochs=2000, name='covid_data')


0 [D loss: 7.159954, acc.: 28.12%] [G loss: 0.011982]
1 [D loss: 0.335417, acc.: 78.12%] [G loss: 7.918631]
2 [D loss: 2.769427, acc.: 43.36%] [G loss: 0.656274]
3 [D loss: 1.262107, acc.: 50.00%] [G loss: 8.956099]
4 [D loss: 1.493726, acc.: 55.47%] [G loss: 1.893862]
5 [D loss: 1.063522, acc.: 49.22%] [G loss: 8.405457]
6 [D loss: 0.866453, acc.: 80.47%] [G loss: 6.782518]
7 [D loss: 0.529113, acc.: 91.02%] [G loss: 1.628308]
8 [D loss: 0.741941, acc.: 73.83%] [G loss: 4.133011]
9 [D loss: 0.610749, acc.: 91.41%] [G loss: 4.563258]
10 [D loss: 0.631758, acc.: 91.80%] [G loss: 0.782184]
11 [D loss: 0.844776, acc.: 60.16%] [G loss: 6.812692]
12 [D loss: 0.896310, acc.: 83.59%] [G loss: 3.987325]
13 [D loss: 0.756626, acc.: 91.41%] [G loss: 0.399868]
14 [D loss: 0.491749, acc.: 77.34%] [G loss: 5.338595]
15 [D loss: 0.470576, acc.: 91.80%] [G loss: 5.302586]
16 [D loss: 0.557269, acc.: 89.45%] [G loss: 0.533932]
17 [D loss: 0.932746, acc.: 56.64%] [G loss: 10.131482]
18 [D loss: 1.2610

## Set target

### Train

In [ ]:
ytrain[ytrain=='Y'] = 0
ytrain[ytrain=='N'] = 1
y_train = np.array(ytrain, dtype = np.int64)
y_train.shape

### Validation

In [ ]:
yval[yval=='Y'] = 0
yval[yval=='N'] = 1
y_val = np.array(yval, dtype = np.int64)
y_val.shape

### Test

In [ ]:
ytest[ytest=='Y'] = 0
ytest[ytest=='N'] = 1
y_test = np.array(ytest, dtype = np.int64)
y_test.shape

## Set class weights

### Train

In [ ]:
ratio_pos = np.count_nonzero(y_train == 0) / len(y_train)
ratio_neg = np.count_nonzero(y_train == 1) / len(y_train)
class_weight_train = np.array((ratio_pos, ratio_neg))
print(class_weight_train)


### Validation

In [ ]:
ratio_pos = np.count_nonzero(y_val == 0) / len(y_val)
ratio_neg = np.count_nonzero(y_val == 1) / len(y_val)
class_weight_val = np.array((ratio_pos, ratio_neg))
print(class_weight_val)


### Test

In [ ]:
ratio_pos = np.count_nonzero(y_test == 0) / len(y_test)
ratio_neg = np.count_nonzero(y_test == 1) / len(y_test)
class_weight_test = np.array((ratio_pos, ratio_neg))
print(class_weight_test)

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True, 
                             featurewise_std_normalization=True, 
                             rotation_range=90)
datagen.fit(X_train)

# Model Definition

## Callbacks

### Checkpoints

In [ ]:
save_dir = os.path.join(
    os.getcwd(),
    '../saved_models'
)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

# This callback saves the weights of the model after each epoch
checkpoint = ModelCheckpoint(
    '../saved_models/weights.epoch_{epoch:02d}.hdf5',
    monitor='val_loss', 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto',
    verbose=1
)

# This callback writes logs for TensorBoard
tensorboard = TensorBoard(
    log_dir='./Graph', 
    histogram_freq=0,  
    write_graph=True
)

# This callback writes logs for TensorBoard
tensorboard_augmented = TensorBoard(
    log_dir='./Graph_augmented', 
    histogram_freq=0,  
    write_graph=True
)

my_callbacks = EarlyStopping(monitor='val_auc', patience=200, verbose=1, mode='max')
my_callbacks_augmented = EarlyStopping(monitor='val_auc', patience=100, verbose=1, mode='max')


callbacks_list = [tensorboard, my_callbacks]
callbacks_list_augmented = [tensorboard_augmented, my_callbacks_augmented]

## Data augmentation

In [ ]:
datagen = ImageDataGenerator(featurewise_center=True, 
                             featurewise_std_normalization=True, 
                             rotation_range=90)
datagen.fit(X_train)

# Data Modeling

## Define CNN structure

In [ ]:
# Fixed
base_model, model = get_model(learning_rate)

# Show layers
print_summary(model)

### Original Data

In [ ]:
# read created data by GANs

import random

previous_val_auc = 0


df = pd.DataFrame()

for n in size_gans:
    print ("Numero imagenes GANs %d" % n)
    
    X_train = np.load(os.path.join(path_images, 'X_train.npy'))
    X_val = np.load(os.path.join(path_images, 'X_val.npy'))
    X_test = np.load(os.path.join(path_images, 'X_test.npy'))
    y_train = np.load(os.path.join(path_images, 'y_train.npy'))
    y_val = np.load(os.path.join(path_images, 'y_val.npy'))
    y_test = np.load(os.path.join(path_images, 'y_test.npy'))
    
    if n != 0:
        X_train_0_gans = np.load(os.path.join(path_gans, 'covid_0_num_imag_%d.npy' % n))
        X_train_1_gans = np.load(os.path.join(path_gans, 'covid_1_num_imag_%d.npy'% n))

        ratio_neg = np.count_nonzero(y_train == 1) / len(y_train)
        size = round(ratio_neg * X_train_1_gans.shape[0])
        lista = range(X_train_1_gans.shape[0]-1)


        ind = random.sample(lista, size)
        X_train_1_gans = X_train_1_gans[ind,:]

        # create target
        y_train_1_gans = np.ones(X_train_1_gans.shape[0], dtype=int)
        y_train_0_gans = np.zeros(X_train_0_gans.shape[0], dtype=int)


        # join data

        X_train_gans = np.concatenate((X_train_1_gans, X_train_0_gans))
        y_train_gans = np.concatenate((y_train_1_gans, y_train_0_gans))

        X_train = np.concatenate((X_train, X_train_gans))
        y_train = np.concatenate((y_train, y_train_gans))
    
    
    # train
    ratio_pos = np.count_nonzero(y_train == 0) / len(y_train)
    ratio_neg = np.count_nonzero(y_train == 1) / len(y_train)
    class_weight_train = np.array((ratio_pos, ratio_neg))

    # val
    ratio_pos = np.count_nonzero(y_val == 0) / len(y_val)
    ratio_neg = np.count_nonzero(y_val == 1) / len(y_val)
    class_weight_val = np.array((ratio_pos, ratio_neg))

    
    out = model.fit(X_train, y_train,
                     validation_data=(X_val, y_val),
                     steps_per_epoch=len(X_train) / batch_size, 
                     epochs=epochs,
                     class_weight=class_weight_train,
                     callbacks = callbacks_list,       
                     verbose=1)
    
    if(len(out.history)):
        acum_tr_auc.append(out.history['auc'][0])
        acum_val_auc.append(out.history['val_auc'][0])
        acum_tr_loss.append(out.history['loss'][0])
        acum_val_loss.append(out.history['val_loss'][0])
                
        acum = pd.DataFrame([acum_tr_auc, acum_val_auc, acum_tr_loss, acum_val_loss])
        acum.to_csv(os.path.join(path_results,'acum_results_no_augmentation.csv'))
                    
        #if len(acum_tr_auc) > 1:
        clear_output()
        best_i = np.argmax(acum_val_auc)
        grafica_entrenamiento(acum_tr_auc, acum_val_auc, acum_tr_loss, acum_val_loss, best_i)
            ### save loss and auc of train and val     
        stopped_epoch = early_stopping.stopped_epoch
        train_loss = out.history['loss'][stopped_epoch-1]
        val_loss = out.history['val_loss'][stopped_epoch-1]
        train_auc = out.history['auc'][stopped_epoch-1]
        val_auc = out.history['val_auc'][stopped_epoch-1]
        model = out.model

        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)

        train_auc = roc_auc_score(y_true = y_train, y_score = pred_train)
        val_auc = roc_auc_score(y_true = y_val, y_score = pred_val)

        
        res = pd.DataFrame([n, epochs, batch_size, stopped_epoch, train_auc, val_auc])
        df = pd.concat([df, res], axis=1)

        df.to_csv(os.path.join(path_results,'model_results_augmentation.csv')) 
                
        model.save(os.path.join(path_results, 'model_augmentation_%d.h5' % n))
            
        if(previous_val_auc < val_auc):
            save_dir = os.path.join(
                    os.getcwd(),
                    '../model_results_y')
            if not os.path.isdir(save_dir):
                    os.makedirs(save_dir)
                model.save(os.path.join(path_results, 'model_no_augmentation_%d.h5' % n))
                
            previous_val_auc = val_auc
                

                           
df.index = ['size_gans', 'epochs', 'batch_size','early_stopping', 'train_auc', 'val_auc']
df.to_csv(os.path.join(path_results,'model_results_augmentation.csv'))  

### Augmented dataset

In [ ]:
# read created data by GANs

import random

previous_val_auc = 0


df = pd.DataFrame()

for n in size_gans:
    print ("Numero imagenes GANs %d" % n)
    
    X_train = np.load(os.path.join(path_images, 'X_train.npy'))
    X_val = np.load(os.path.join(path_images, 'X_val.npy'))
    X_test = np.load(os.path.join(path_images, 'X_test.npy'))
    y_train = np.load(os.path.join(path_images, 'y_train.npy'))
    y_val = np.load(os.path.join(path_images, 'y_val.npy'))
    y_test = np.load(os.path.join(path_images, 'y_test.npy'))
    
    if n != 0:
        X_train_0_gans = np.load(os.path.join(path_gans, 'covid_0_num_imag_%d.npy' % n))
        X_train_1_gans = np.load(os.path.join(path_gans, 'covid_1_num_imag_%d.npy'% n))

        ratio_neg = np.count_nonzero(y_train == 1) / len(y_train)
        size = round(ratio_neg * X_train_1_gans.shape[0])
        lista = range(X_train_1_gans.shape[0]-1)


        ind = random.sample(lista, size)
        X_train_1_gans = X_train_1_gans[ind,:]

        # create target
        y_train_1_gans = np.ones(X_train_1_gans.shape[0], dtype=int)
        y_train_0_gans = np.zeros(X_train_0_gans.shape[0], dtype=int)


        # join data

        X_train_gans = np.concatenate((X_train_1_gans, X_train_0_gans))
        y_train_gans = np.concatenate((y_train_1_gans, y_train_0_gans))

        X_train = np.concatenate((X_train, X_train_gans))
        y_train = np.concatenate((y_train, y_train_gans))
    
    
    # train
    ratio_pos = np.count_nonzero(y_train == 0) / len(y_train)
    ratio_neg = np.count_nonzero(y_train == 1) / len(y_train)
    class_weight_train = np.array((ratio_pos, ratio_neg))

    # val
    ratio_pos = np.count_nonzero(y_val == 0) / len(y_val)
    ratio_neg = np.count_nonzero(y_val == 1) / len(y_val)
    class_weight_val = np.array((ratio_pos, ratio_neg))

    datagen = ImageDataGenerator(featurewise_center=True, 
                             featurewise_std_normalization=True,
                             rotation_range=90,
                             brightness_range = (0.25, 0.75))
    datagen.fit(X_train)
    
    base_model_augmented, model_augmented = get_model(learning_rate)
    
    out = model_augmented.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size, seed=seed),
                     validation_data=(X_val, y_val),
                     steps_per_epoch=len(X_train) / batch_size, 
                     epochs=epochs,
                     class_weight=class_weight_train,
                     callbacks = callbacks_list,       
                     verbose=1)
    
    if(len(out.history)):
        acum_tr_auc.append(out.history['auc'][0])
        acum_val_auc.append(out.history['val_auc'][0])
        acum_tr_loss.append(out.history['loss'][0])
        acum_val_loss.append(out.history['val_loss'][0])
                
        acum = pd.DataFrame([acum_tr_auc, acum_val_auc, acum_tr_loss, acum_val_loss])
        acum.to_csv(os.path.join(path_results,'acum_results_no_augmentation.csv'))
                    
        #if len(acum_tr_auc) > 1:
        clear_output()
        best_i = np.argmax(acum_val_auc)
        grafica_entrenamiento(acum_tr_auc, acum_val_auc, acum_tr_loss, acum_val_loss, best_i)
            ### save loss and auc of train and val     
        stopped_epoch = early_stopping.stopped_epoch
        train_loss = out.history['loss'][stopped_epoch-1]
        val_loss = out.history['val_loss'][stopped_epoch-1]
        train_auc = out.history['auc'][stopped_epoch-1]
        val_auc = out.history['val_auc'][stopped_epoch-1]
        model = out.model

        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)

        train_auc = roc_auc_score(y_true = y_train, y_score = pred_train)
        val_auc = roc_auc_score(y_true = y_val, y_score = pred_val)

        
        res = pd.DataFrame([n, epochs, batch_size, stopped_epoch, train_auc, val_auc])
        df = pd.concat([df, res], axis=1)

        df.to_csv(os.path.join(path_results,'model_results_augmentation.csv')) 
                
        model.save(os.path.join(path_results, 'model_augmentation_%d.h5' % n))
            
        if(previous_val_auc < val_auc):
            save_dir = os.path.join(
                    os.getcwd(),
                    '../model_results_y')
            if not os.path.isdir(save_dir):
                    os.makedirs(save_dir)
                model.save(os.path.join(path_results, 'model_no_augmentation_%d.h5' % n))
                
            previous_val_auc = val_auc
                

                           
df.index = ['size_gans', 'epochs', 'batch_size','early_stopping', 'train_auc', 'val_auc']
df.to_csv(os.path.join(path_results,'model_results_augmentation.csv'))  

### Predict

In [ ]:
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)
pred_test = model.predict(X_test)

### Metrics

In [ ]:
auc_train = roc_auc_score(y_true = y_train, y_score = pred_train)
auc_val = roc_auc_score(y_true = y_val, y_score = pred_val)
auc_test = roc_auc_score(y_true = y_test, y_score = pred_test)
print('AUC train = %s - AUC val = %s - AUC test = %s' % (str(auc_train), str(auc_val), str(auc_test)))

In [ ]:
y_labels_train = (pred_train >= 0.5).astype(int)
y_labels_val = (pred_val >= 0.5).astype(int)
y_labels_test = (pred_test >= 0.5).astype(int)
cm_train = confusion_matrix(y_pred = y_labels_train, y_true = y_train)
cm_val = confusion_matrix(y_pred = y_labels_val, y_true = y_val)
cm_test = confusion_matrix(y_pred = y_labels_test, y_true = y_test)
print(cm_train)
print(cm_val)
print(cm_test)

In [ ]:

fpr_train, tpr_train, threshold_train = roc_curve(y_train, pred_train)
roc_auc_train = roc_auc_score(y_true = y_train, y_score = pred_train)
fpr_val, tpr_val, threshold_val = roc_curve(y_val, pred_val)
roc_auc_val = roc_auc_score(y_true = y_val, y_score = pred_val)
fpr_test, tpr_test, threshold_test = roc_curve(y_test, pred_test)
roc_auc_test = roc_auc_score(y_true = y_test, y_score = pred_test)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'r', label = 'AUC = %0.2f' % roc_auc_test)
plt.plot(fpr_val, tpr_val, 'g', label = 'AUC = %0.2f' % roc_auc_val)
plt.plot(fpr_test, tpr_test, 'b', label = 'AUC = %0.2f' % roc_auc_test)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'b', label = 'AUC = %0.2f' % roc_auc_train)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Save output

In [ ]:
X_train.to_csv(os.path.join('../predictions_y', 'X_train.csv'))
np.savetxt(os.path.join('../predictions_y', 'predictions_train.csv'), pred_train, delimiter=";")
np.savetxt(os.path.join('../predictions_y', 'y_train.csv'), y_train, delimiter=";")
X_val.to_csv(os.path.join('../predictions_y', 'X_val.csv'))
np.savetxt(os.path.join('../predictions_y', 'predictions_val.csv'), pred_val, delimiter=";")
np.savetxt(os.path.join('../predictions_y', 'y_val.csv'), y_val, delimiter=";")
X_test.to_csv(os.path.join('../predictions_y', 'X_test.csv'))
np.savetxt(os.path.join('../predictions_y', 'predictions_test.csv'),pred_test, delimiter=";")
np.savetxt(os.path.join('../predictions_y', 'y_test.csv'), y_test, delimiter=";")

# Results

## AUC confidence intervals

In [ ]:
AUC_CI_train = bootstrap_auc(y_train, pred_train, bootstraps = 100, fold_size = 1000,)
AUC_CI_val = bootstrap_auc(y_val, pred_val, bootstraps = 100, fold_size = 1000)
AUC_CI_test = bootstrap_auc(y_test, pred_test, bootstraps = 100, fold_size = 1000,)
AUC_CI = print_confidence_intervals(AUC_CI_train,)
AUC_CI = AUC_CI.append(print_confidence_intervals(AUC_CI_val), ignore_index=True)
AUC_CI = AUC_CI.append(print_confidence_intervals(AUC_CI_test), ignore_index=True)
AUC_CI.index = ['Train', 'Val', 'Test'];
AUC_CI

## Plot AUC

In [ ]:
fpr_train, tpr_train, threshold_train = roc_curve(y_train, pred_train)
roc_auc_train = roc_auc_score(y_true = y_train, y_score = pred_train)
fpr_val, tpr_val, threshold_val = roc_curve(y_val, pred_val)
roc_auc_val = roc_auc_score(y_true = y_val, y_score = pred_val)
fpr_test, tpr_test, threshold_test = roc_curve(y_test, pred_test)
roc_auc_test = roc_auc_score(y_true = y_test, y_score = pred_test)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr_train, tpr_train, 'r', label = 'AUC = %0.2f' % roc_auc_train)
plt.plot(fpr_val, tpr_val, 'g', label = 'AUC = %0.2f' % roc_auc_val)
plt.plot(fpr_test, tpr_test, 'b', label = 'AUC = %0.2f' % roc_auc_test)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Heatmap / Gradcam

In [ ]:
for i in np.where(y_test == 1)[0]:
    print('index ' + str(i));
    heat_map, superimposed_img = show_heatmap(model, X_test[i], y_test[i], pred_test[i])